<a href="https://colab.research.google.com/github/Neo87z/Essay-Geneartion-ElutherAI-GPT-NEO-/blob/main/Essay-Geneartion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydub
!pip install pyngrok
!pip install vosk
!pip install praat-parselmouth
!pip install pronouncing
!pip install pyrebase
!pip install flask_ngrok
!pip install firebase
!pip install -U flask-cors
!pip install sentence-splitter
!pip install SentencePiece
!pip install python_jwt
!pip install gcloud
!pip install sseclient
!pip install pycrypto
!pip install requests-toolbelt
!pip install bert-extractive-summarizer
!pip install transformers
!pip install spacy 
!pip install wordnet
!pip install numpy 
!pip install matplotlib
!pip install seaborn 
!pip install sklearn 
!pip install happytransformer
!pip install datetime
! pip install sentence-splitter
! pip install SentencePiece 
!pip install pyinflect


In [ ]:
from pydub import AudioSegment
from flask import *
from flask_ngrok import run_with_ngrok
import os
import json
import math
import vosk
import librosa
import numpy
from firebase import firebase
import pandas as pd
from pydub import AudioSegment
from parselmouth.praat import call
import pronouncing
import parselmouth 
from parselmouth.praat import call
import pyrebase
from pydub import AudioSegment
from flask_cors import CORS
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import pipeline
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from string import punctuation
from heapq import nlargest
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import spacy
from nltk.corpus import wordnet
import random
from datetime import datetime
import time
from happytransformer import HappyTextToText
from datasets import load_dataset
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pyinflect
from transformers import pipeline
import re

In [ ]:
Sentence=""
def UpdateData(x):
  print(x)
  
  global Sentence


  Sentence=x



In [ ]:
model_name = 'tuner007/pegasus_paraphrase'
torch_device =  'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=1660,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
def GetPharasedSentence(text):
  data =get_response(text, 2)
  return(data[1])


In [ ]:
GetPharasedSentence("Because once a leader is found  and I don't mean just a leader in a group of soldiers but someone who is important to the group  it can make a big difference.")

In [ ]:
nltk.download('wordnet')
happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")#Grammer Checking Model 
modelXYZ= GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")#GTP Neo model
tokenizerXYZ = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

In [ ]:
from happytransformer import TTSettings#Grammer checking
beam_settings =  TTSettings(num_beams=15, min_length=1, max_length=1200)
result = happy_tt.generate_text("Because once a leadersss is found  and I don't mean just a leader in a group of soldiers but someone who is important to the group  it can make a big difference in their performance or in how they carry out their mission.", args=beam_settings)
print(result.text)            

Because once a leader is found, and I don't mean just a leader in a group of soldiers, but someone who is important to the group, it can make a big difference in their performance or in how they carry out their mission.


In [ ]:
nlp = spacy.load('en_core_web_sm')
nltk.download('omw-1.4')      
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words  

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
FBConn= firebase.FirebaseApplication('https://stek-e122c-default-rtdb.firebaseio.com/',None)#Database Connection

In [ ]:
def GenerateDraft8(Text):#To Generate a Draft using GTP NEo model
  prompt =Text

  input_ids = tokenizerXYZ(prompt, return_tensors="pt").input_ids

  gen_tokens = modelXYZ.generate(
      input_ids,
      do_sample=True,
      temperature=0.7,
      max_length=500,
  )
  
  gen_text = tokenizerXYZ.batch_decode(gen_tokens)[0]
  newTex=gen_text.split()
  Data=""
  for i in newTex:
    Data=Data+" "+ i
  return Data

In [ ]:
print(GenerateDraft8("Why graduating from high school is important"))

In [ ]:
from sentence_splitter import SentenceSplitter, split_text_into_sentences #To seperate Sentences using . 

splitter = SentenceSplitter(language='en')



In [ ]:
FBConn= firebase.FirebaseApplication('https://stek-e122c-default-rtdb.firebaseio.com/',None)

In [ ]:
def getTopic(): #Function to get Essay Topic in the firebase
   result = FBConn.get('/data/', '')
   return result['SelectedTopic']



In [ ]:
def mainProgramme(text):
  try:
    Subject=get_subject_phrase(nlp(text))#Get the Subject Of the sentenbce
    last =getlast(text)#Get the Last of the sentence
    newSub=str(Subject)#Converting Subject to String
    Descision=getMatching(newSub.lower(),last)#Getting the Mtahcing Data from the Database
    if(Descision == "None"):#checkiong if there is any matching essay data
      finalData=GenerateDraft8(text)
      BeforePrint=finalData#Assinging The daft to a variable
      finalData=finalData.split('.')#Spliting the Draft into a list 
      toPrint=''
      for x in range(0,len(finalData)):#Looping the splitted list
        print(x)
        toPrint=toPrint+" "+GetPharasedSentence(finalData[x])#Makking the sentences more meangnfull using peagus model
      sentence_list = splitter.split(toPrint)#Splitting the new Optmized Essay
      afterGrammar=''
      for x in range (0,len(sentence_list)):#Looping the optmizteds Essay
        TempData=sentence_list[x].split(" ")#
        if(len(TempData)> 5):#checking if the senetces word is greater than 5
          result = happy_tt.generate_text(sentence_list[x], args=beam_settings)#Fuxing the if there is any grammer misatkes
          afterGrammar=afterGrammar+ " "+ result.text#combinign the new steneces
      FinalOutput=afterGrammar
      FinalOutput=FinalOutput.replace("\ "," ")#Removing unwated characters
      DataTSendTO = splitter.split(FinalOutput)
      DataTSendTO = list(dict.fromkeys(DataTSendTO))
      FinalPrint=""
      for x in range (0,len(DataTSendTO)):
        FinalPrint=FinalPrint+" "+ DataTSendTO[x]
      AddToDatabase(FinalPrint,newSub.strip(),last,BeforePrint)#Addning to the Databas
      data ={#Makung a JSon object to returb 
          "FinalChoiceData" : FinalPrint,
          "Orgiginal":BeforePrint,
      }
      return data#Returingin Dtaa
    else:#if there is a essay in the databse
      print()
      data ={
          "FinalChoiceData" : Descision['FinalChoiceData'],
          "Orgiginal":Descision['Orgiginal'],
      }
      return data
  except:    
      data ={
          "FinalChoiceData" : "Please Generate Again",
          "Orgiginal":BeforePrint,
      }
      return data





In [ ]:
def AddToDatabase(text,subject,last,Orgi):#function to Save In the firerbase Db
  FirebaseDataStore={
    "TopicSubject":subject.lower(),
    "TopicLast":last.lower(),
    "Text":text,
    "Original":Orgi
  }
  result = FBConn.get('/ResearchTopiCData/', '')  
  result=FBConn.post('/ResearchTopiCData/',FirebaseDataStore)


In [ ]:
def get_subject_phrase(doc):# Function to gte the subject in the setence
    for token in doc:
        if ("subj" in token.dep_):
            subtree = list(token.subtree)
            start = subtree[0].i
            end = subtree[-1].i + 1
            return doc[start:end]

In [ ]:
def getlast(text): #function to gte the last in the sentence
  newText=text.split()
  last=(len(newText))
  return newText[last-1]

In [ ]:
def getMatching(TopicSubject,TopicLast):#to get the matching Data in the databse
  try:
    TopicSubject.lower()#Conveting the subject to Lowrcase
    TopicLast.lower()#Converting the Topic Last to Lower case
    NewSubject=TopicSubject.strip()#Removing whitpasce
    
    NewLast=TopicLast.strip()#removing whitpasce
    
    TopicLastNew=NewLast.lower()
   
    TopicSubjectNew=NewSubject.lower()
   
    SynoTopicSubject=GetSynonyms(TopicSubjectNew)#getting to the synonims for the SUbject
    
    SynoTopicLast=GetSynonyms(TopicLastNew)#getting to the synonims for the Last
    
    UniqueTopicSubject=unique(SynoTopicSubject)#Getting unique Names
    UniqueTopicSubject.append(NewSubject)
    UniqueTopicLast=unique(SynoTopicLast)     
   
    result = FBConn.get('/ResearchTopiCData/', '')  
    
    for key in result.keys():#Looping the database
      
      for data2 in UniqueTopicSubject:

        
               
        if(result[key]['TopicSubject'].lower() == data2): 
          for data in UniqueTopicLast:
            if(result[key]['TopicLast'].lower() ==data):
              print(result[key]['Original'])
              data ={
                  "FinalChoiceData" : result[key]['Text'],
                  "Orgiginal":result[key]['Original'],
              }
              return data
    return "None"
  except:
    
    return "None"


In [ ]:
def GetSynonyms(word):#Function to get the Synonims 

  synonyms=[]
  for syn in wordnet.synsets(word):
    for lemma in syn.lemmas():
      synonyms.append(lemma.name()  )
  return synonyms

In [ ]:
def unique(list1):#function ot gte the unieque of the list
    unique_list = [] 
    for x in list1:
        if x not in unique_list:
            unique_list.append(x)
    return (unique_list)

In [ ]:
from flask import *
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask_cors import CORS


In [ ]:
app = Flask(__name__)
cors = CORS(app)

In [ ]:
! ngrok authtoken 2HT7xP7fNrb8xB0G6TsoY1wupuH_3qvtaDBFuEC8KKzHnjYkf

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
checkSum=True;

In [ ]:
@app.route("/GetEssay",methods=['POST'])
def GetEssay():
  global checkSum
  if(checkSum == True):
    checkSum=False
    data=mainProgramme(Sentence)
    my_str=data['FinalChoiceData']
    my_str2=data['Orgiginal']
    dataXXX={
          "FinalChoiceData":my_str,
          "Original":my_str2
    }
    checkSum=True
    return (dataXXX)

In [ ]:
@app.route("/SaveSentnece",methods=['POST'])
def SaveSentnece():
  Data=reqData=request.get_json()
  UpdateData(Data['SentenceData'])
  
  return ("data")

In [ ]:
run_with_ngrok(app)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b657-34-125-240-11.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [14/Nov/2022 01:42:34] "OPTIONS /SaveSentnece HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Nov/2022 01:42:34] "POST /SaveSentnece HTTP/1.1" 200 -


he pros and cons of online retail


...

 the pros and cons of online retail The pros and cons of online retail are numerous. First, online retail is a form of retailing. Second, it is a relatively new business model. Third, it has tremendous potential to disrupt retail. Fourth, it is a relatively new space for the internet. Online retail is a form of retailing. In the retail industry, online retail is a form of retailing. In the online retail industry, it’s a form of retailing. In essence, the online and offline ae not at all related. The offline retail space is a physical space, a store. The online space is the online space. The offline space is the physical space, the physical store. In my opinion, online retail is a form of retailing. Online retail is the most profitable form of retailing. First, online retail is a form of retailing. in the retail space, onlinse retail is a form of retailing. In the online space, it’s a form of retailing. The online retail space is not the same thing as the offline retail space. The offli

INFO:werkzeug:127.0.0.1 - - [14/Nov/2022 02:17:54] "OPTIONS /SaveSentnece HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Nov/2022 02:17:55] "POST /SaveSentnece HTTP/1.1" 200 -


Sources of alternative energy


INFO:werkzeug:127.0.0.1 - - [14/Nov/2022 02:17:57] "POST /GetEssay HTTP/1.1" 200 -


 Sourcess of alternativ energy in the United States Alternative energy sources exist in the U.S. Despite the fact that the U.S. produces far more energy per capita than other industrialized countries, the U.S. is unable to generate its own alternative energy. U.S. energy imports come mainly from two sources: hydroelectric power and coal power. Hydropower is the most common source of U.S. energy. Hydropower The primary power source in the U.S. is hydroelectric power. In most of the U.S. states, a dam or lake is the primary power source. In the South, the sources of power are the Mississippi River River  and the Colorado River. In the West, the sources of power are the Colorado River and the Black River in Colorado. In the U.S. the power is produced by damming rivers, and using wind or solar energy to create electricity. In the early years, the power was available from a small number of power plants built at the turn of the 20th century to meet the needs of the growing middle class. Sinc